Try to use pipelis and messy dataframe

In [112]:

import re
from gensim.models import Word2Vec

import pandas as pd
import numpy as np
import itertools

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline 
from sklearn.compose import ColumnTransformer
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.model_selection import learning_curve, validation_curve, train_test_split, KFold, StratifiedKFold, cross_val_score, GridSearchCV, RandomizedSearchCV, cross_validate, RepeatedStratifiedKFold
from sklearn.linear_model import Perceptron, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import FunctionTransformer, StandardScaler, OneHotEncoder, OrdinalEncoder, MinMaxScaler
from sklearn.datasets import fetch_openml
from sklearn.ensemble import RandomForestClassifier

from scipy.stats import loguniform, beta, uniform

from mlxtend.feature_selection import SequentialFeatureSelector as SFS

from imblearn.over_sampling import SMOTE, RandomOverSampler
from imblearn.pipeline import Pipeline as IMBPipeline

import missingno as msno

import matplotlib.pyplot as plt
import warnings


In [113]:
df = pd.read_csv('movies-3.csv')

In [114]:
df.isnull().sum(axis = 0)

id                           0
title                        4
genres                  210628
original_language            0
overview                118438
popularity                   0
production_companies    385352
release_date             52009
budget                       0
revenue                      0
runtime                  34403
status                       0
tagline                 614322
vote_average                 0
vote_count                   0
credits                 224946
keywords                512169
poster_path             184895
backdrop_path           499779
recommendations         687805
dtype: int64

In [115]:
#funciton to add missing values, required by teacher

def add_missing(col, amount):
    X = col.copy()
    size = amount if amount >= 1 else int(len(X) * amount)
    indexes = np.random.choice(len(X), size, replace = False )
    X[indexes] = np.nan
    return X

def add_missing_rows(df, amount):
    X = df.copy()
    rows, cols = X.shape
    size = amount if amount >= 1 else int(rows * amount)
    indexes = np.random.choice(rows, size, replace = False ) + 0.5
    for i in indexes:
        X.loc[i] = np.full((cols,),np.nan)
    X = X.sort_index().reset_index(drop=True)
    return X

In [116]:
#add missing values
df['original_language'] = add_missing(df['original_language'], 0.3)
df['popularity'] = add_missing(df['popularity'], 0.25)
df['budget'] = add_missing(df['budget'], 0.32)
df['vote_average'] = add_missing(df['vote_average'], 0.42)
df['vote_count'] = add_missing(df['vote_count'], 0.1)

In [117]:
df.to_csv('movies_corrupted_df.csv')

In [118]:
df_m = pd.read_csv('movies_corrupted_df.csv')

In [119]:
df_m.isnull().sum(axis = 0)

Unnamed: 0                   0
id                           0
title                        4
genres                  210628
original_language       216777
overview                118438
popularity              180647
production_companies    385352
release_date             52009
budget                  231228
revenue                      0
runtime                  34403
status                       0
tagline                 614322
vote_average            303487
vote_count               72259
credits                 224946
keywords                512169
poster_path             184895
backdrop_path           499779
recommendations         687805
dtype: int64

In [120]:
df_m.head()

,Unnamed: 0,id,title,genres,original_language,overview,popularity,production_companies,release_date,budget,...,runtime,status,tagline,vote_average,vote_count,credits,keywords,poster_path,backdrop_path,recommendations
0,0,615656,Meg 2: The Trench,Action-Science Fiction-Horror,en,An exploratory dive into the deepest depths of...,8763.998,Apelles Entertainment-Warner Bros. Pictures-di...,2023-08-02,129000000.0,...,116.0,Released,Back for seconds.,7.079,1365.0,Jason Statham-Wu Jing-Shuya Sophia Cai-Sergio ...,based on novel or book-sequel-kaiju,/4m1Au3YkjqsxF8iwQy0fPYSxE0h.jpg,/qlxy8yo5bcgUw2KAmmojUKp4rHd.jpg,1006462-298618-569094-1061181-346698-1076487-6...
1,1,758323,The Pope's Exorcist,Horror-Mystery-Thriller,en,Father Gabriele Amorth Chief Exorcist of the V...,5953.227,Screen Gems-2.0 Entertainment-Jesus & Mary-Wor...,2023-04-05,18000000.0,...,103.0,Released,Inspired by the actual files of Father Gabriel...,7.433,545.0,Russell Crowe-Daniel Zovatto-Alex Essoe-Franco...,spain-rome italy-vatican-pope-pig-possession-c...,/9JBEPLTPSm0d1mbEcLxULjJq9Eh.jpg,/hiHGRbyTcbZoLsYYkO4QiCLYe34.jpg,713704-296271-502356-1076605-1084225-1008005-9...
2,2,667538,Transformers: Rise of the Beasts,Action-Adventure-Science Fiction,en,When a new threat capable of destroying the en...,5409.104,Skydance-Paramount-di Bonaventura Pictures-Bay...,2023-06-06,NaN,...,127.0,Released,Unite or fall.,7.340,1007.0,Anthony Ramos-Dominique Fishback-Luna Lauren V...,peru-alien-end of the world-based on cartoon-b...,/gPbM0MK8CP8A174rmUwGsADNYKD.jpg,/woJbg7ZqidhpvqFGGMRhWQNoxwa.jpg,496450-569094-298618-385687-877100-598331-4628...
3,3,640146,Ant-Man and the Wasp: Quantumania,Action-Adventure-Science Fiction,en,Super-Hero partners Scott Lang and Hope van Dy...,4425.387,Marvel Studios-Kevin Feige Productions,2023-02-15,200000000.0,...,125.0,Released,Witness the beginning of a new dynasty.,6.507,2811.0,Paul Rudd-Evangeline Lilly-Jonathan Majors-Kat...,hero-ant-sequel-superhero-based on comic-famil...,/qnqGbB22YJ7dSs4o6M7exTpNxPz.jpg,/m8JTwHFwX7I7JY5fPe4SjqejWag.jpg,823999-676841-868759-734048-267805-965839-1033...
4,4,677179,Creed III,Drama-Action,en,After dominating the boxing world Adonis Creed...,NaN,Metro-Goldwyn-Mayer-Proximity Media-Balboa Pro...,2023-03-01,NaN,...,116.0,Released,You can't run from your past.,NaN,1129.0,Michael B. Jordan-Tessa Thompson-Jonathan Majo...,philadelphia pennsylvania-husband wife relatio...,/cvsXj3I9Q2iyyIo95AecSd1tad7.jpg,/5i6SjyDbDWqyun8klUuCxrlFbyw.jpg,965839-267805-943822-842942-1035806-823999-107...


In [121]:
#dropping all df with language <= 100
mask = df_m['original_language'].value_counts() >= 1000
df_m= df_m[df['original_language'].isin(mask[mask].index)]

In [122]:
df_m['original_language'].value_counts()

en    265520
fr     31263
es     29751
de     25844
ja     19389
pt     14314
ru     12329
it     12012
zh     11701
ko      6601
hi      4518
cs      4275
nl      4177
sv      4152
tr      3574
ar      3462
tl      3266
pl      3176
cn      2998
ml      2847
ta      2739
el      2674
da      2559
id      2275
xx      2250
fa      2068
hu      1857
fi      1757
te      1590
bn      1470
no      1434
th      1265
he      1090
Name: original_language, dtype: int64

In [123]:
df_m.drop_duplicates(subset=['title','release_date'], inplace=True)

In [124]:
#we drop the column we don't need
df_no = df_m.drop(['id', 'tagline', 'release_date','poster_path', 'backdrop_path', 'recommendations'], axis=1)

In [125]:
df_no.head()

,Unnamed: 0,title,genres,original_language,overview,popularity,production_companies,budget,revenue,runtime,status,vote_average,vote_count,credits,keywords
0,0,Meg 2: The Trench,Action-Science Fiction-Horror,en,An exploratory dive into the deepest depths of...,8763.998,Apelles Entertainment-Warner Bros. Pictures-di...,129000000.0,352056482.0,116.0,Released,7.079,1365.0,Jason Statham-Wu Jing-Shuya Sophia Cai-Sergio ...,based on novel or book-sequel-kaiju
1,1,The Pope's Exorcist,Horror-Mystery-Thriller,en,Father Gabriele Amorth Chief Exorcist of the V...,5953.227,Screen Gems-2.0 Entertainment-Jesus & Mary-Wor...,18000000.0,65675816.0,103.0,Released,7.433,545.0,Russell Crowe-Daniel Zovatto-Alex Essoe-Franco...,spain-rome italy-vatican-pope-pig-possession-c...
2,2,Transformers: Rise of the Beasts,Action-Adventure-Science Fiction,en,When a new threat capable of destroying the en...,5409.104,Skydance-Paramount-di Bonaventura Pictures-Bay...,NaN,407045464.0,127.0,Released,7.340,1007.0,Anthony Ramos-Dominique Fishback-Luna Lauren V...,peru-alien-end of the world-based on cartoon-b...
3,3,Ant-Man and the Wasp: Quantumania,Action-Adventure-Science Fiction,en,Super-Hero partners Scott Lang and Hope van Dy...,4425.387,Marvel Studios-Kevin Feige Productions,200000000.0,475766228.0,125.0,Released,6.507,2811.0,Paul Rudd-Evangeline Lilly-Jonathan Majors-Kat...,hero-ant-sequel-superhero-based on comic-famil...
4,4,Creed III,Drama-Action,en,After dominating the boxing world Adonis Creed...,NaN,Metro-Goldwyn-Mayer-Proximity Media-Balboa Pro...,NaN,269000000.0,116.0,Released,NaN,1129.0,Michael B. Jordan-Tessa Thompson-Jonathan Majo...,philadelphia pennsylvania-husband wife relatio...


In [126]:
#drop empty genres and overview 
df_no.drop_duplicates(inplace=True)
index = df_no[(df_no['genres']=='') & (df_no['overview']=='') & (df_no['credits']=='') & (df_no['keywords']=='') & (df_no['production_companies']=='') & (df_no['title']=='')].index
df_no.drop(index, inplace=True)

In [127]:
df_no.isnull().sum(axis = 0)

Unnamed: 0                   0
title                        1
genres                  138894
original_language            0
overview                 76981
popularity              114890
production_companies    249448
budget                  147237
revenue                      0
runtime                  22661
status                       0
vote_average            193158
vote_count               46006
credits                 148193
keywords                326761
dtype: int64

In [128]:
def get_sentence_embedding(sentence, model):
    words = re.findall(r'\w+', sentence.lower())
    vectors = [model.wv[word] for word in words if word in model.wv]
    return sum(vectors) / len(vectors) if vectors else [0] * 100

In [129]:
class TextExtraction(BaseEstimator, TransformerMixin):
    def __init__(self, column):
        self.column = column

    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        corpus = X[self.column].apply(lambda x: re.findall(r'\w+', str(x).lower()))
        word2vec_model = Word2Vec(corpus, vector_size=100, window=5, min_count=1, workers=4)
        return pd.DataFrame(X[self.column].apply(lambda x: get_sentence_embedding(str(x), word2vec_model)).tolist())

    def fit_transform(self, X, y=None):
        return self.transform(X, y)

In [130]:
df_no = df_no.dropna(subset=['genres'], axis=0)
df_no = df_no.dropna(subset=['overview'], axis=0)
df_no = df_no.dropna(subset=['credits'], axis=0)
df_no = df_no.dropna(subset=['keywords'], axis=0)
df_no = df_no.dropna(subset=['production_companies'], axis=0)
df_no = df_no.dropna(subset=['title'], axis=0)


In [131]:
df_no = df_no.dropna(subset=['genres'], axis=0)
df_no['genres']  = [i.split('-')[0] for i in df_no['genres']]

In [132]:
pipe_popularity = Pipeline(
    [   
        ('Most frequesnt',SimpleImputer(strategy='most_frequent')),
        ('StandarScaler', StandardScaler(with_mean = False))

    ]
)

pipe_budget_revenue = Pipeline(
    [
        ('Mean',SimpleImputer(strategy='mean')),
        ('MinMax', StandardScaler(with_mean = True))
        

    ]
)

pipe_voteAvg = Pipeline(
    [
        ('knn', KNNImputer(n_neighbors=5)),
        ('minmax', MinMaxScaler())
    ]
)

pipe_votecount = Pipeline(
    [
        ('Mean',SimpleImputer(strategy='median')),
        ('StandarScaler', StandardScaler(with_mean = False))
    ]
)

pipe_overview = Pipeline(
    [
        ('text', TextExtraction(column='overview'))
    ])

pipe_credits = Pipeline(
    [
        ('text', TextExtraction(column='credits'))
    ])

pipe_cproduction = Pipeline(
    [
        ('text', TextExtraction(column='production_companies'))
    ])



trf = ColumnTransformer(
    transformers=[
        ('popularity', pipe_popularity, ['popularity']),
        ('budget_revenue', pipe_budget_revenue, ['budget', 'revenue']),
        ('vote_avg', pipe_voteAvg, ['vote_average']),
        ('vote_count', pipe_votecount, ['vote_count']),
        ('original_language', OneHotEncoder(), ['original_language']),
        ('status', OneHotEncoder(), ['status']),
        ('overview', pipe_overview, ['overview']),
        ('credits', pipe_credits, ['credits']),
        ('production', pipe_cproduction, ['production_companies'])
    ]
)

WE use a sample df just to make it faster and fix errors

In [133]:
sampled_df = df_no.sample(n=3000, random_state=42)
sampled_df.reset_index(drop=True, inplace=True)

In [134]:
sampled_df.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [147]:
X = sampled_df.drop('genres', axis=1)  # Features

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(sampled_df['genres'])

In [148]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify = y, random_state=30, shuffle=True)

In [149]:
#come il prof, applico il columntransformer con ttute le pipelines insieme a i vari ML algorithms

model_pipeline = IMBPipeline([
    ('trans', trf),
    ('sampler', SMOTE()),
    ('dim_reduction', PCA(n_components=0.8)),
    ('classifier', Perceptron())
])

In [150]:
model_pipeline.fit(X_train, y_train)

Pipeline(steps=[('trans',
                 ColumnTransformer(transformers=[('popularity',
                                                  Pipeline(steps=[('Most '
                                                                   'frequesnt',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('StandarScaler',
                                                                   StandardScaler(with_mean=False))]),
                                                  ['popularity']),
                                                 ('budget_revenue',
                                                  Pipeline(steps=[('Mean',
                                                                   SimpleImputer()),
                                                                  ('MinMax',
                                                                   StandardScaler())]),
                                                  ['budget', 'revenue']),
                                                 ('vote_avg',
                                                  Pipeline(ste...
                                                  Pipeline(steps=[('text',
                                                                   TextExtraction(column='overview'))]),
                                                  ['overview']),
                                                 ('credits',
                                                  Pipeline(steps=[('text',
                                                                   TextExtraction(column='credits'))]),
                                                  ['credits']),
                                                 ('production',
                                                  Pipeline(steps=[('text',
                                                                   TextExtraction(column='production_companies'))]),
                                                  ['production_companies'])])),
                ('sampler', SMOTE()), ('dim_reduction', PCA(n_components=0.8)),
                ('classifier', Perceptron())])

In [152]:
model_pipeline.predict(X_test)

array([11, 14, 14, 14, 14, 14, 11, 14, 14, 14, 14, 11, 14, 14, 14, 14, 11,
       11, 11, 11, 11, 11, 14, 14, 16, 11, 11, 14, 11, 11, 11, 11, 14, 14,
       14, 14, 11, 11, 11, 11, 11, 11, 11, 14,  0, 11, 11, 14, 11, 14, 11,
       11, 14, 14, 14, 14, 14, 14, 11, 11, 14, 14, 11, 11, 14, 14, 14, 11,
       14, 11, 14, 11, 14, 14, 14, 11, 14, 11, 11, 14, 11, 14, 14, 11, 14,
       14, 11, 14, 11, 11, 14, 14, 11,  4, 11, 14, 11, 11, 14, 11, 11, 11,
       14, 11, 11, 11, 11, 14, 14, 14, 14, 11, 14, 14,  4, 14, 14, 11, 14,
       14, 14, 14, 11, 11, 14, 11, 11, 14, 11, 14, 11, 14, 11, 14, 11, 11,
       11, 14, 11, 11, 14, 11, 11, 14, 11, 11, 14, 11, 14, 11, 14, 14, 11,
       14, 14, 14, 11, 11, 14, 14, 11, 14, 14, 14, 14, 14, 14, 14, 11, 14,
       11, 11, 14, 11, 14, 11, 11, 14, 11, 11, 11, 11, 14, 14, 11, 11, 11,
       14, 14, 14, 11, 14, 11, 11, 14, 11, 14, 11, 14, 11, 11, 14, 14, 11,
       11, 11, 11, 11, 11, 14, 14, 14, 14, 11, 14, 14, 11, 11, 14, 11, 14,
       11, 11, 11, 11, 11

In [153]:
sampler_configs = [
    {
        'sampler':[None],# The element is bypassed
    },
    {
        'sampler':[SMOTE(n_jobs=100)],
        'sampler__sampling_strategy':['minority', 1.2, 0.9, 0.7]
    },
    {
        'sampler':[RandomOverSampler()],
        'sampler__sampling_strategy':['minority', 1.2, 0.9, 0.7]
    }
]

dim_reduction_configs = [
    {
        'dim_reduction': [None]
    },
    {
        'dim_reduction': [PCA()],
        'dim_reduction__n_components': [0.5, 0.7, 0.9]
    },
    {
        'dim_reduction': [LDA()]
    },
    {
        'dim_reduction': [SFS(estimator=Perceptron(), cv = None, scoring = 'rand_score')],
        'dim_reduction__estimator': [Perceptron(), LogisticRegression()],
        'dim_reduction__k_features' : [5,7,10]  
    }
]

classifier_configs = [
    {
        'classifier__eta0' : loguniform(0.001,100),
        'classifier': [Perceptron()] ,
        'classifier__max_iter': [100, 200, 500, 3000] ,
        'classifier__class_weight' : [None, 'balanced']

    },
    {
        'classifier': [LogisticRegression(solver='saga')],
        'classifier__C' : loguniform(0.001,100),
        'classifier__penalty': ['l1','l2'],
        'classifier__class_weight' : [None, 'balanced']

    },
    {
        'classifier': [KNeighborsClassifier()],
        'classifier__n_neighbors': [3,5,7,9]
    },
    {
        'classifier' : [RandomForestClassifier()],
        'classifier__n_estimators' : [10,50,100, 500]
    }
]

In [154]:
all_configs = [dict(itertools.chain(*(e.items() for e in configuration))) for configuration in itertools.product(sampler_configs,dim_reduction_configs,classifier_configs)]

In [155]:
f'Number of all possible configurations: {len(all_configs)}'

'Number of all possible configurations: 48'

In [156]:
all_configs[:2]

[{'sampler': [None],
  'dim_reduction': [None],
  'classifier__eta0': <scipy.stats._distn_infrastructure.rv_continuous_frozen at 0x181689e90>,
  'classifier': [Perceptron()],
  'classifier__max_iter': [100, 200, 500, 3000],
  'classifier__class_weight': [None, 'balanced']},
 {'sampler': [None],
  'dim_reduction': [None],
  'classifier': [LogisticRegression(solver='saga')],
  'classifier__C': <scipy.stats._distn_infrastructure.rv_continuous_frozen at 0x18168b790>,
  'classifier__penalty': ['l1', 'l2'],
  'classifier__class_weight': [None, 'balanced']}]

In [157]:
rs = RandomizedSearchCV(model_pipeline,
    param_distributions=all_configs,
    n_iter=1000,
    n_jobs=100,
    cv = 2,
    scoring='rand_score'
)

In [ ]:
scores = cross_validate(rs, X_train, y_train, scoring='rand_score', cv = 5, return_estimator=True, verbose=3) #ERROR